In [1]:
import numpy as np
import pandas as pd
import gc
import os
PATH = '.'
print(os.listdir(f'{PATH}'))

['sample_submission.csv', 'data', 'periods_test.csv', 'periods_train.csv', 'test_active.csv', 'test.csv', 'train_active.csv', 'train.csv', '.ipynb_checkpoints', 'best.hdf5', 'train.ipynb', 'submit_nn_p3_0.2260.csv']


In [2]:
df_train = pd.read_csv(f'{PATH}/train.csv')
df_test = pd.read_csv(f'{PATH}/test.csv')

df_y_train = df_train['deal_probability']
df_x_train = df_train.drop(['deal_probability'], axis=1)

In [3]:
df_x_train.head(10)


,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1
0,b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0
1,2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0
2,ba83aefab5dc,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0
3,02996f1dd2ea,bf5cccea572d,Татарстан,Набережные Челны,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Автокресло,Продам кресло от0-25кг,2200.0,286,2017-03-25,Company,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,796.0
4,7c90be56d2ab,ef50846afc0b,Волгоградская область,Волгоград,Транспорт,Автомобили,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",Все вопросы по телефону.,40000.0,3,2017-03-16,Private,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,2264.0
5,51e0962387f7,bbfad0b1ad0a,Татарстан,Чистополь,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Авто люлька,В хорошем состоянии,1300.0,9,2017-03-28,Private,eb6ad1231c59d3dc7e4020e724ffe8e4d302023ddcbb99...,796.0
6,c4f260a2b48a,08f469d2e6f7,Нижегородская область,Нижний Новгород,Для дома и дачи,Ремонт и строительство,Сантехника и сауна,NaN,NaN,Водонагреватель 100 литров нержавейка плоский,Электро водонагреватель накопительный на 100 л...,11000.0,125,2017-03-23,Private,0330f6ac561f5db1fa8226dd5e7e127b5671d44d075a98...,2823.0
7,6b71309d6a8a,fef86baa002c,Пермский край,Пермь,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Джинсы,26,Бойфренды colins,Бойфренды в хорошем состоянии.,500.0,61,2017-03-25,Private,9bab29a519e81c14f4582024adfebd4f11a4ac71d323a6...,567.0
8,c5b969cb63a2,055825270190,Оренбургская область,Оренбург,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Платья и юбки,> 50 (XXL),Платье,54 раз мер очень удобное,500.0,85,2017-03-17,Private,75ce06d1f939a31dfb2af8ac55f08fa998fa336d13ee05...,415.0
9,b1570962e68c,f9e8f831d94c,Нижегородская область,Нижний Новгород,Личные вещи,Детская одежда и обувь,Для девочек,Обувь,25,Полу ботиночки замш натур.Бамбини,По стельке 15.5см мерить приокский район. Цвет...,400.0,136,2017-03-22,Company,54fb8521135fda77a860bfd2fac6bf46867ab7c06796e3...,46.0


In [4]:
df_x_train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1503424 entries, 0 to 1503423
Data columns (total 17 columns):
item_id                 1503424 non-null object
user_id                 1503424 non-null object
region                  1503424 non-null object
city                    1503424 non-null object
parent_category_name    1503424 non-null object
category_name           1503424 non-null object
param_1                 1441848 non-null object
param_2                 848882 non-null object
param_3                 640859 non-null object
title                   1503424 non-null object
description             1387148 non-null object
price                   1418062 non-null float64
item_seq_number         1503424 non-null int64
activation_date         1503424 non-null object
user_type               1503424 non-null object
image                   1390836 non-null object
image_top_1             1390836 non-null float64
dtypes: float64(2), int64(1), object(14)
memory usage: 195.0+ MB


In [5]:
df_test.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 508438 entries, 0 to 508437
Data columns (total 17 columns):
item_id                 508438 non-null object
user_id                 508438 non-null object
region                  508438 non-null object
city                    508438 non-null object
parent_category_name    508438 non-null object
category_name           508438 non-null object
param_1                 485528 non-null object
param_2                 275209 non-null object
param_3                 202107 non-null object
title                   508438 non-null object
description             508438 non-null object
price                   477853 non-null float64
item_seq_number         508438 non-null int64
activation_date         508438 non-null object
user_type               508438 non-null object
image                   465829 non-null object
image_top_1             465829 non-null float64
dtypes: float64(2), int64(1), object(14)
memory usage: 65.9+ MB


In [6]:
# viewing # of unique value in each column 
for col in df_x_train.columns:
    print(col, len(df_x_train[col].unique()))

item_id 1503424
user_id 771769
region 28
city 1733
parent_category_name 9
category_name 47
param_1 372
param_2 272
param_3 1220
title 788377
description 1317103
price 17007
item_seq_number 28232
activation_date 21
user_type 3
image 1390837
image_top_1 3063


In [7]:
df_x_train['image_top_1'].fillna(value=3067, inplace=True)
df_test['image_top_1'].fillna(value=3067, inplace=True)


In [8]:
df_x_train['param_1'].fillna(value='_NA_', inplace=True)
df_test['param_1'].fillna(value='_NA_', inplace=True)

df_x_train['param_2'].fillna(value='_NA_', inplace=True)
df_test['param_2'].fillna(value='_NA_', inplace=True)

df_x_train['param_3'].fillna(value='_NA_', inplace=True)
df_test['param_3'].fillna(value='_NA_', inplace=True)

In [9]:
df_x_train['description'].fillna(value='_NA_', inplace=True)
df_test['description'].fillna(value='_NA_', inplace=True)

In [10]:
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
import argparse

#create config init
config = argparse.Namespace()


/home/antor/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [11]:
def tknzr_fit(col, df_trn, df_test):
    tknzr = Tokenizer(filters='', lower=False, split=' ', oov_token='oov' )
    tknzr.fit_on_texts(df_trn[col])
    return np.array(tknzr.texts_to_sequences(df_trn[col])), np.array(tknzr.texts_to_sequences(df_test[col])), tknzr

In [12]:
def to_categorical_idx(col, df_trn, df_test):
    merged = pd.concat([df_trn[col], df_test[col]])
    train_size = df_trn[col].shape[0]
    idxs, uniques = pd.factorize(merged)
    
    return idxs[:train_size], idxs[train_size:], uniques

In [13]:
df_x_train['param_1'].unique()
tr_reg, te_reg, tknzr_reg = to_categorical_idx('region', df_x_train, df_test)


In [14]:
tr_reg, te_reg, tknzr_reg    = to_categorical_idx('region', df_x_train, df_test)
tr_pcn, te_pcn, tknzr_pcn    = to_categorical_idx('parent_category_name', df_x_train, df_test)
tr_cn, te_cn, tknzr_cn       = to_categorical_idx('category_name', df_x_train, df_test)
tr_ut, te_ut, tknzr_ut       = to_categorical_idx('user_type', df_x_train, df_test)
tr_city, te_city, tknzr_city = to_categorical_idx('city', df_x_train, df_test)

tr_p1, te_p1, tknzr_p1 = to_categorical_idx('param_1', df_x_train, df_test)
tr_p2, te_p2, tknzr_p2 = to_categorical_idx('param_2', df_x_train, df_test)
tr_p3, te_p3, tknzr_p3 = to_categorical_idx('param_3', df_x_train, df_test)

In [15]:
tr_week = pd.to_datetime(df_x_train['activation_date']).dt.weekday.astype(np.int32).values
te_week = pd.to_datetime(df_test['activation_date']).dt.weekday.astype(np.int32).values
tr_week = np.expand_dims(tr_week, axis=-1)
te_week = np.expand_dims(te_week, axis=-1)

In [16]:
tr_imgt1 = df_x_train['image_top_1'].astype(np.int32).values
te_imgt1 = df_test['image_top_1'].astype(np.int32).values
tr_imgt1 = np.expand_dims(tr_imgt1, axis=-1)
te_imgt1 = np.expand_dims(te_imgt1, axis=-1)

In [17]:
eps = 1e-10
tr_price = np.log(df_x_train['price']+eps)
te_price = np.log(df_test['price']+eps)
tr_price[tr_price.isna()] = -1.
te_price[te_price.isna()] = -1.

tr_price = np.expand_dims(tr_price, axis=-1)
te_price = np.expand_dims(te_price, axis=-1)

In [18]:
tr_itemseq = np.log(df_x_train['item_seq_number'])
te_itemseq = np.log(df_test['item_seq_number'])
#tr_itemseq = df_x_train['item_seq_number']
#te_itemseq = df_test['item_seq_number']
# price_tr[price_tr.isna()] = -1.
# price_te[price_te.isna()] = -1.

tr_itemseq = np.expand_dims(tr_itemseq, axis=-1)
te_itemseq = np.expand_dims(te_itemseq, axis=-1)

In [19]:
config.len_desc = 100000


In [20]:
from keras.preprocessing.sequence import pad_sequences


In [21]:
tknzr_desc = Tokenizer(num_words=config.len_desc, lower=True)


In [22]:
tknzr_desc.fit_on_texts(df_x_train['description'].values)


In [ ]:
tr_desc_seq = tknzr_desc.texts_to_sequences(df_x_train['description'].values)
te_desc_seq = tknzr_desc.texts_to_sequences(df_test['description'].values)


In [ ]:
config.maxlen= 256


In [ ]:
tr_desc_pad = pad_sequences(tr_desc_seq, maxlen=config.maxlen)
te_desc_pad = pad_sequences(te_desc_seq, maxlen=config.maxlen)


In [ ]:
gc.collect()


In [ ]:
## categorical
config.len_reg   = len(tknzr_reg)
config.len_pcn   = len(tknzr_pcn)
config.len_cn    = len(tknzr_cn) 
config.len_ut    = len(tknzr_ut)
config.len_city  = len(tknzr_city) +1
config.len_week  = 7
config.len_imgt1 = int(df_x_train['image_top_1'].max())+1
config.len_p1    = len(tknzr_p1)+1
config.len_p2    = len(tknzr_p2)+1
config.len_p3    = len(tknzr_p3)+1

## continuous
config.len_price   = 1
config.len_itemseq = 1


In [ ]:
## categorical
max_emb = 64
config.emb_reg   = min(max_emb,(config.len_reg   + 1)//2)
config.emb_pcn   = min(max_emb,(config.len_pcn   + 1)//2)
config.emb_cn    = min(max_emb,(config.len_cn    + 1)//2)
config.emb_ut    = min(max_emb,(config.len_ut    + 1)//2)
config.emb_city  = min(max_emb,(config.len_city  + 1)//2)
config.emb_week  = min(max_emb,(config.len_week  + 1)//2)
config.emb_imgt1 = min(max_emb,(config.len_imgt1 + 1)//2)
config.emb_p1    = min(max_emb,(config.len_p1    + 1)//2)
config.emb_p2    = min(max_emb,(config.len_p2    + 1)//2)
config.emb_p3    = min(max_emb,(config.len_p3    + 1)//2)

#continuous
config.emb_price   = 16
config.emb_itemseq = 16

#text
config.emb_desc = 256

In [ ]:
valid_idx = df_y_train.sample(frac=0.2, random_state=1991).index
train_idx = df_y_train[np.invert(df_y_train.index.isin(valid_idx))].index

In [ ]:
[print(k.shape) for k in [tr_reg, tr_pcn, tr_cn, tr_ut, tr_city, tr_week, tr_imgt1, tr_p1, tr_p2, tr_p3, tr_price, tr_itemseq]]
X      = np.array([tr_reg, tr_pcn, tr_cn, tr_ut.squeeze(), tr_city, tr_week.squeeze(), tr_imgt1.squeeze(), tr_p1, tr_p2, tr_p3, tr_price.squeeze(), tr_itemseq.squeeze()])
X_test = np.array([te_reg, te_pcn, te_cn, te_ut.squeeze(), te_city, te_week.squeeze(), te_imgt1.squeeze(), te_p1, te_p2, te_p3, te_price.squeeze(), te_itemseq.squeeze()])

Y = df_y_train

In [ ]:
tr_reg

In [ ]:
X_train = [x[train_idx] for x in X]
X_valid = [x[valid_idx] for x in X]
X_test = [x for x in X_test]

Y_train = Y[train_idx]
Y_valid = Y[valid_idx]

In [ ]:
X_train.append(tr_desc_pad[train_idx])
X_valid.append(tr_desc_pad[valid_idx])
X_test.append(te_desc_pad)

In [ ]:
gc.collect()


In [ ]:
from keras.layers import Input, Embedding, Dense, BatchNormalization, Activation, Dropout
from keras.layers import GlobalMaxPool1D, GlobalMaxPool2D
from keras.layers import concatenate, Flatten
from keras.layers import LSTM, CuDNNGRU, CuDNNLSTM, GRU
from keras.models import Model
from keras.utils import multi_gpu_model

from keras import backend as K

from keras.optimizers import RMSprop, Adam, SGD
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

from keras_contrib.layers.normalization import InstanceNormalization

### rmse loss for keras
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true))) 

In [ ]:
config.batch_size = 4096


In [ ]:
def get_model():
    do = 0.30
    K.clear_session()
    inp_reg = Input(shape=(1, ), name='inp_region')
    emb_reg = Embedding(config.len_reg, config.emb_reg, name='emb_region')(inp_reg)
    
    inp_pcn = Input(shape=(1, ), name='inp_parent_category_name')
    emb_pcn = Embedding(config.len_pcn, config.emb_pcn, name='emb_parent_category_name')(inp_pcn)

    inp_cn = Input(shape=(1, ), name='inp_category_name')
    emb_cn = Embedding(config.len_cn, config.emb_cn, name="emb_category_name" )(inp_cn)
    
    inp_ut = Input(shape=(1, ), name='inp_user_type')
    emb_ut = Embedding(config.len_ut, config.emb_ut, name='emb_user_type' )(inp_ut)
    
    inp_city = Input(shape=(1, ), name='inp_city')
    emb_city = Embedding(config.len_city, config.emb_city, name='emb_city' )(inp_city)

    inp_week = Input(shape=(1, ), name='inp_week')
    emb_week = Embedding(config.len_week, config.emb_week, name='emb_week' )(inp_week)

    inp_imgt1 = Input(shape=(1, ), name='inp_imgt1')
    emb_imgt1 = Embedding(config.len_imgt1, config.emb_imgt1, name='emb_imgt1')(inp_imgt1)
    
    inp_p1 = Input(shape=(1, ), name='inp_p1')
    emb_p1 = Embedding(config.len_p1, config.emb_p1, name='emb_p1')(inp_p1)
    
    inp_p2 = Input(shape=(1, ), name='inp_p2')
    emb_p2 = Embedding(config.len_p2, config.emb_p2, name='emb_p2')(inp_p2)
    
    inp_p3 = Input(shape=(1, ), name='inp_p3')
    emb_p3 = Embedding(config.len_p3, config.emb_p3, name='emb_p3')(inp_p3)
    
    conc_cate = concatenate([emb_reg, emb_pcn,  emb_cn, emb_ut, emb_city, emb_week, emb_imgt1, emb_p1, emb_p2, emb_p3], axis=-1, name='concat_categorcal_vars')
    conc_cate = Flatten()(conc_cate)
    
    inp_price = Input(shape=(1, ), name='inp_price')
    emb_price = inp_price#InstanceNormalization()(inp_price)
    #emb_price = Dense(config.emb_price, activation='tanh', name='emb_price')(inp_price)

    inp_itemseq = Input(shape=(1, ), name='inp_itemseq')
    emb_itemseq = inp_itemseq# InstanceNormalization()(inp_itemseq)
    emb_itemseq = Dense(config.emb_itemseq, activation='tanh', name='emb_itemseq')(emb_itemseq)
    
#    conc_cont = concatenate([conc_cate, emb_price, emb_itemseq], axis=-1)
    conc_cont = concatenate([conc_cate, emb_price, emb_itemseq], axis=-1)
    x = Dense(512)(conc_cont)
    #x = BatchNormalization()(x)
    x = Activation(activation='relu')(x)
    
    x = Dense(256)(x)
    #x = BatchNormalization()(x)
    x = Activation(activation='relu')(x)
    x = Dropout(do)(x)

    x = Dense(128)(x)
   # x = BatchNormalization()(x)
    x = Activation(activation='relu')(x)
    x = Dropout(do)(x)

    ### text
    inp_desc = Input(shape=(config.maxlen, ), name='inp_desc')
    emb_desc = Embedding(config.len_desc, config.emb_desc, name='emb_desc')(inp_desc)
    
    desc_layer = CuDNNGRU(128, return_sequences=True)(emb_desc)
    desc_layer = CuDNNGRU(128, return_sequences=True)(desc_layer)
    desc_layer = CuDNNGRU(128, return_sequences=True)(desc_layer)
    desc_layer = CuDNNGRU(128, return_sequences=False)(desc_layer)

    conc_desc = concatenate([x, desc_layer], axis=-1)
    conc_desc = Dense(128)(conc_desc)
    #conc_desc = BatchNormalization()(conc_desc)
    conc_desc = Activation(activation='relu')(conc_desc)
    
    conc_desc = Dense(64)(conc_desc)
    #conc_desc = BatchNormalization()(conc_desc)
    conc_desc = Activation(activation='relu')(conc_desc)
    conc_desc = Dropout(do)(conc_desc)

    conc_desc = Dense(32)(conc_desc)
    #conc_desc = BatchNormalization()(conc_desc)
    conc_desc = Activation(activation='relu')(conc_desc)
    conc_desc = Dropout(do)(conc_desc)

###

    outp = Dense(1, activation='sigmoid', name='output')(conc_desc)

    model = Model(inputs = [inp_reg, inp_pcn, inp_cn, inp_ut, inp_city, inp_week, inp_imgt1, inp_p1, inp_p2, inp_p3,
                            inp_price, inp_itemseq, inp_desc], outputs = outp)
    return model


In [ ]:
model = get_model()
model.summary()
model = multi_gpu_model(model, gpus=2)
model.compile(optimizer=Adam(lr=1e-3, amsgrad=True), loss = root_mean_squared_error, metrics=[root_mean_squared_error])
# model.compile(optimizer=RMSprop(lr=0.0005, decay=0.00001), loss = root_mean_squared_error, metrics=['mse', root_mean_squared_error])


In [ ]:
### callbacks
checkpoint = ModelCheckpoint('best.hdf5', monitor='val_loss', verbose=1, save_best_only=True)
early = EarlyStopping(patience=5, mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-7, verbose=1, mode='min')


In [ ]:
model.fit(x=X_train, y=Y_train, validation_data=(X_valid, Y_valid), batch_size=config.batch_size, epochs=100, callbacks=[checkpoint,early,reduce_lr], verbose=1)

In [ ]:
pred = model.predict(X_test)

subm = pd.read_csv(f'{PATH}/sample_submission.csv')
subm['deal_probability'] = pred
subm.to_csv('submit_{}_{:.4f}.csv'.format('nn_p3', 0.226), index=False)